In [ ]:
#по поводу прошлого ДЗ писали, что ничего стращного, если сдать до утра, надеюсь с этим тоже...

In [55]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm 
from collections import defaultdict, Counter


#!pip install nltk
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Данные

In [4]:
! wget --no-check-certificate https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Luxury_Beauty.json.gz
! gzip -d Luxury_Beauty.json.gz

--2022-12-17 19:54:46--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Luxury_Beauty.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 73193215 (70M) [application/x-gzip]
Saving to: ‘Luxury_Beauty.json.gz’

Luxury_Beauty.json. 100%[===================>]  69.80M   101MB/s    in 0.7s    

2022-12-17 19:54:47 (101 MB/s) - ‘Luxury_Beauty.json.gz’ saved [73193215/73193215]



In [56]:
data = pd.read_json("Luxury_Beauty.json", lines=True)
data

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,2,3,True,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,1276560000,NaN,NaN
1,5,14,True,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,1262822400,NaN,NaN
2,5,NaN,True,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN
3,5,NaN,True,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN
4,5,NaN,True,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,1523923200,{'Size:': ' 3.5 oz.'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
574623,5,NaN,True,"03 20, 2017",AHYJ78MVF4UQO,B01HIQEOLO,Lori Fox,Great color and I prefer shellac over gel,Five Stars,1489968000,NaN,NaN
574624,5,NaN,True,"10 26, 2016",A1L2RT7KBNK02K,B01HIQEOLO,Elena,Best shellac I have ever used. It doesn't tak...,Best shellac I have ever used,1477440000,NaN,NaN
574625,5,NaN,True,"09 30, 2016",A36MLXQX9WPPW9,B01HIQEOLO,Donna D. Harris,Great polish and beautiful color!!,Great polish!,1475193600,NaN,NaN
574626,1,2,True,"12 5, 2016",A23DRCOMC2RIXF,B01HJ2UY0W,Y.Y. Chen,"The perfume is good, but the spray head broke ...",Spray head broke off within a month,1480896000,"{'Size:': ' 1.7 Fluid Ounce', 'Color:': ' Multi'}",NaN


In [57]:
data = data.sample(100000, random_state=1000)

In [58]:
corpus = data["reviewText"]
for line in corpus.sample(3):
  print(line)
  print("\n\n\n")

This is my second order. I notice a difference on my hand where I mix the serum enforce applying it to my face. People say I have nice skin so there is probably some effect on my face




It isn't worth the money. I noticed no difference using this product.




This is the best edge gel I've ever used.  I love that it isn't too greasy, long lasting and I only use a little bit  I use it like every other day.  I will be ordering more!!!!!!






## Метаданные

In [10]:
! wget --no-check-certificate https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Luxury_Beauty.json.gz
! gzip -d meta_Luxury_Beauty.json.gz

--2022-12-17 19:55:17--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Luxury_Beauty.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 7608964 (7.3M) [application/x-gzip]
Saving to: ‘meta_Luxury_Beauty.json.gz’

meta_Luxury_Beauty. 100%[===================>]   7.26M  --.-KB/s    in 0.1s    

2022-12-17 19:55:18 (65.8 MB/s) - ‘meta_Luxury_Beauty.json.gz’ saved [7608964/7608964]



In [59]:
metadata = pd.read_json("meta_Luxury_Beauty.json", lines=True)
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[After a long day of handling thorny situation...,,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,"[B00GHX7H0A, B00FRERO7G, B00R68QXCS, B000Z65AZ...",,,[],"4,324 in Beauty & Personal Care (","[B00FRERO7G, B00GHX7H0A, B07GFHJRMX, B00TJ3NBN...",{' Product Dimensions: ': '2.2 x 2.2 ...,Luxury Beauty,,NaT,$30.00,B00004U9V2,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[If you haven't experienced the pleasures of b...,,AHAVA Bath Salts,[],,,[],"1,633,549 in Beauty & Personal Care (",[],{' Product Dimensions: ': '3 x 3.5 x ...,Luxury Beauty,,NaT,,B0000531EN,[],[]
2,[],,"[Rich, black mineral mud, harvested from the b...",,"AHAVA Dead Sea Mineral Mud, 8.5 oz, Pack of 4",[],,,[],"1,806,710 in Beauty &amp; Personal Care (",[],{' Product Dimensions: ': '5.1 x 3 x ...,Luxury Beauty,,NaT,,B0000532JH,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[This liquid soap with convenient pump dispens...,,"Crabtree &amp; Evelyn Hand Soap, Gardeners, 10...",[],,,[],[],"[B00004U9V2, B00GHX7H0A, B00FRERO7G, B00R68QXC...",{' Product Dimensions: ': '2.6 x 2.6 ...,Luxury Beauty,,NaT,$15.99,B00005A77F,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Remember why you love your favorite blanket? ...,,Soy Milk Hand Crme,"[B000NZT6KM, B001BY229Q, B008J724QY, B0009YGKJ...",,,[],"42,464 in Beauty &amp; Personal Care (",[],{' Product Dimensions: ': '7.2 x 2.2 ...,Luxury Beauty,,NaT,$18.00,B00005NDTD,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [60]:
metadata[metadata["category"].apply(lambda x: bool(x))]

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes


# Задание 1
Предложите 3 способа найти упоминания товаров в отзывах



1.   **Bootstrapping**: можно написать шаблоны типа ["bottle of X", "tube of X", ...], ["hand _NOUN_, face _NOUN_, ...] и найти упоминания товаров. (+использовать семантическую близость и словари синонимов и найти синонимы)
2.   Специфично для данных: из графы **summary** можно выделить существительные, отранжировать их по **TF-iDF** (idf из всего корпуса). (+использовать семантическую близость и словари синонимов и найти синонимы)
3. Можно использовать **метаданные**, получить названия товаров из отзывов, найти пересечения слов из названия и слов в отзыве
4. Самим обучить **sequence tagging** (но нужны размеченные данные)



# Задание 2 и 3 
Реализуйте один из предложенных вами способов. Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед).

### Мэтчинг с метадатой
Для скорости пришлось взять сжмпл из 1000 первых товаров

In [74]:
def preprocess(text, join=False):
  if join:
    return " ".join([w for w in word_tokenize(text.lower()) if w.isalpha()])
  return [w for w in word_tokenize(text.lower()) if w.isalpha()]

In [75]:
product_reviews = defaultdict(list)

for product_i in tqdm(metadata.iloc[:1000].index):

  product_name = metadata["title"].iloc[product_i]
  product_name = preprocess(product_name, join=True)
  # можно было бы добавить product_category и это бы сильно помогло, но они в данных все пустые :(
  #key = " ".join([])

  asin = metadata["asin"][product_i]
  reviews = data[data["asin"] == asin]["reviewText"].apply(lambda x: preprocess(str(x))).tolist()
  #reviews = data[data["asin"] == asin].index.tolist()
  product_reviews[product_name] = reviews


len(product_reviews)

  0%|          | 0/1000 [00:00<?, ?it/s]

771

In [77]:
entities_w_right_neighbours = defaultdict(Counter)
entities_w_left_neighbours = defaultdict(Counter)

def _get_neighbors(center, tokens):
  left = []
  right = []

  if center in tokens:
    for i, t in enumerate(tokens):
      if t == center and not t in stop_words:
        
        #left
        if i > 0 and not tokens[i - 1] in stop_words:
          left.append(tokens[i - 1])

        #right
        if i < len(tokens) - 1 and not tokens[i + 1] in stop_words:
          right.append(tokens[i + 1])
  return left, right


for pname in tqdm(product_reviews):
  for review in product_reviews[pname]:
    for pname_w in pname.split():
      left, right = _get_neighbors(pname_w, review)
      if right:
        entities_w_right_neighbours[pname_w].update(right)
      if left:
        entities_w_left_neighbours[pname_w].update(left)

  0%|          | 0/771 [00:00<?, ?it/s]

In [86]:
#Задание 5 до коллокаций
nes = ["cream", "toner", "soap", "deodorant", "cleanser"]
for ne in nes:
  print(ne)
  print("-----")
  for n, _ in entities_w_left_neighbours[ne].most_common(3):
    print(" ".join([n, ne]))
  for n, _ in entities_w_right_neighbours[ne].most_common(3):
    print(" ".join([ne,  n]))
  print()

    
  #поэтому по идее надо бы бграммы...

cream
-----
shaving cream
shave cream
eye cream
cream ever
cream works
cream smells

toner
-----
use toner
best toner
soothing toner
toner helps
toner came
toner smells

soap
-----
shave soap
shaving soap
fabulous soap
soap works
soap could
soap hope

deodorant
-----
best deodorant
worst deodorant
ms deodorant
deodorant ever
deodorant keeps
deodorant spray

cleanser
-----
facial cleanser
face cleanser
skin cleanser
cleanser seem
cleanser expensive
cleanser remove



# Задания 4 и 5
Ранжируйте n-граммы с помощью 3 коллокационных метрик. Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.

In [92]:
entity_to_neighbour = defaultdict(Counter)
neighbour_to_entity = defaultdict(Counter)

for key in set(entities_w_left_neighbours.keys())|set(entities_w_right_neighbours.keys()):
  entity_to_neighbour[key] = entities_w_left_neighbours[key] + entities_w_right_neighbours[key]
  for neighbour, count in entity_to_neighbour[key].items():
    neighbour_to_entity[neighbour].update({key: count})


#total_word_count
twc = sum([len(v) for v in product_reviews.values()])

In [93]:
entity_to_neighbour["cream"]["shaving"], neighbour_to_entity["shaving"]["cream"]

(252, 252)

In [95]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

def score_bigram(w1, w2,
                 scoring_function,
                 entity_to_neighbour = entity_to_neighbour,
                 neighbour_to_entity = neighbour_to_entity,
                 n_xx = twc):
  
  n_ii = entity_to_neighbour[w1][w2]
  n_ix = sum(entity_to_neighbour[w1].values())
  n_xi = sum(neighbour_to_entity[w2].values())
  return scoring_function(n_ii, (n_ix, n_xi), n_xx)

print("%0.4f" % score_bigram("shaving", "cream", bigram_measures.pmi))

3.0695


In [104]:
def score_all_bigrams(scoring_function,
                      entity_to_neighbour, 
                      neighbour_to_entity,
                      twc):
  bigram_scores = defaultdict(Counter)

  for w1 in entity_to_neighbour:
    for w2 in entity_to_neighbour[w1]:
      score = score_bigram(w1, w2,
                           scoring_function,
                           entity_to_neighbour = entity_to_neighbour,
                           neighbour_to_entity = neighbour_to_entity,
                           n_xx = twc)
      bigram_scores[w1][w2] = score

  return bigram_scores

def print_best_ngrams_by_entity(entity, scores, top_n = 5,
                                entities_w_left_neighbours = entities_w_left_neighbours,
                                entities_w_right_neighbours = entities_w_right_neighbours):
  print(entity)
  print("---")
  best_n = scores[entity].most_common(top_n)
  for neighbour, _ in best_n:
    left = entities_w_right_neighbours[entity][neighbour]
    right = entities_w_right_neighbours[entity][neighbour]
    if left >= right:
      print(" ".join([neighbour, entity]))
    else:
      print(" ".join([entity, neighbour]))
  print()


In [105]:
#pmi

pmi_scores = score_all_bigrams(bigram_measures.pmi, 
                               entity_to_neighbour, 
                               neighbour_to_entity,
                               twc)
for entity in nes:
  print_best_ngrams_by_entity(entity, pmi_scores)

cream
---
cuticle cream
cold cream
fortified cream
sented cream
shaven cream

toner
---
afterward toner
soothing toner
came toner
helps toner
mint toner

soap
---
westie soap
generates soap
runs soap
pump soap
butter soap

deodorant
---
crappy deodorant
ms deodorant
axe deodorant
excels deodorant
build deodorant

cleanser
---
dermo cleanser
foams cleanser
remove cleanser
seem cleanser
algae cleanser



In [107]:
#likelihood

likelihood_scores = score_all_bigrams(bigram_measures.likelihood_ratio, 
                               entity_to_neighbour, 
                               neighbour_to_entity,
                               twc)
for entity in nes:
  print_best_ngrams_by_entity(entity, likelihood_scores)

cream
---
shaving cream
shave cream
hand cream
eye cream
products cream

toner
---
soothing toner
came toner
use toner
afterward toner
helps toner

soap
---
westie soap
generates soap
runs soap
pump soap
butter soap

deodorant
---
ms deodorant
worst deodorant
stick deodorant
best deodorant
free deodorant

cleanser
---
facial cleanser
gentle cleanser
nioxin cleanser
moisturizing cleanser
dermo cleanser



In [108]:
#student_t

student_t_scores = score_all_bigrams(bigram_measures.student_t, 
                               entity_to_neighbour, 
                               neighbour_to_entity,
                               twc)
for entity in nes:
  print_best_ngrams_by_entity(entity, student_t_scores)

cream
---
shaving cream
shave cream
eye cream
hand cream
moisturizing cream

toner
---
use toner
best toner
soothing toner
came toner
helps toner

soap
---
shave soap
shaving soap
westie soap
generates soap
runs soap

deodorant
---
best deodorant
ms deodorant
worst deodorant
stick deodorant
free deodorant

cleanser
---
facial cleanser
gentle cleanser
face cleanser
skin cleanser
nioxin cleanser



In [109]:
#chi_sq

chi_sq_scores = score_all_bigrams(bigram_measures.chi_sq, 
                               entity_to_neighbour, 
                               neighbour_to_entity,
                               twc)
for entity in nes:
  print_best_ngrams_by_entity(entity, chi_sq_scores)

cream
---
shaving cream
shave cream
hand cream
eye cream
antiwrinkle cream

toner
---
afterward toner
soothing toner
came toner
helps toner
use toner

soap
---
westie soap
generates soap
runs soap
pump soap
butter soap

deodorant
---
ms deodorant
crappy deodorant
axe deodorant
excels deodorant
build deodorant

cleanser
---
facial cleanser
gentle cleanser
dermo cleanser
nioxin cleanser
remove cleanser



Итого: результаты разных скорингов очень похожи: выдают много того, что может быть итоговой неодносложной сущностью, мало оценочых слов. В целом примеры сильно пересекаются. В 4 задании нужно выбрать лучшую скоринговую функцию -- это сложно сделать по таким данным -- лучше бы взять все товары и все отзывы (это было очень долго), но не факт что результаты поменяются, и нужно проверить, если сами сущности могут быть би- и триграммами.

Так как есть я бы взяла student_t или likelihood ratio, потому что там больше всего по сравнению с дргуими функциями оценочных прилагательных у дезодоранта.